In [ ]:
from Chemometrics.basic import read_dx
import matplotlib.pyplot as plt
import glob
import re
import numpy as np
import pandas as pd
from Chemometrics.basic import regstr
%matplotlib inline

In [ ]:
dxs = glob.glob('./T20原始图谱-科迈恩/*.dx')

In [ ]:
samples = map(lambda x:regstr(x,'(?<=\\\).+(?=\s)'),dxs)#断言：前缀，后缀
samples = list(samples)

In [ ]:
dfs = pd.concat([read_dx(dxsi).reset_index(drop=True) for dxsi in dxs],axis=1,ignore_index=True)
dfs.columns = samples
dfs.index = read_dx(dxs[0]).index

In [ ]:
mdfs = dfs.groupby(dfs.index).mean()#将相同波数的吸光度平均

是否将同一个样品平行测定看成不同样品？  
有的厂家只有一个样品，所以是。

In [ ]:
names_sample = [samplei[0] for samplei in samples]

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(mdfs.T,names_sample)
clf.score(mdfs.T,names_sample)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=10)
pca.fit(mdfs.T)
X = pca.transform(mdfs.T)

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf,X,names_sample,cv=3)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
a = le.fit_transform(names_sample)
plt.scatter(X[:,0],X[:,1],c=a,cmap='tab20')
plt.colorbar()

In [ ]:
from sklearn.svm import SVC
clf = SVC()
cross_val_score(clf,X,names_sample,cv=3)

In [ ]:
pd.Series(names_sample).to_excel('tw20_IRlabel.xlsx')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

# 预处理
- 针对变量
- 针对观测

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
#去掉只有一个样本的组
X_origin = mdfs.T
X_origin.loc[:,'label']= names_sample
X3 = X_origin.groupby('label').filter(lambda x:len(x)>3)
label = X3.loc[:,'label']
X3 = X3.drop('label',axis=1)

In [ ]:
#UV标度化
sd = StandardScaler(with_mean=False)
Xuv = sd.fit_transform(X3.T).T # 针对观测
Xuv_feature = sd.fit_transform(X3) #针对变量

In [ ]:
#参数优化
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
knn = KNeighborsClassifier()
parameters = {'n_neighbors':np.arange(1,11)}
clf = GridSearchCV(knn,parameters,cv=6)
clf.fit(Xuv,label)
clf.best_score_,clf.best_params_

In [ ]:
clf.fit(Xuv_feature,label)
clf.best_score_,clf.best_params_